In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
from collections import Counter
import itertools
import copy

In [3]:
# Load matrix data
all_combinations_4000 = pd.read_csv('all_combinations_4000.csv', index_col=0)
all_combinations1_4000 = pd.read_csv('all_combinations1_4000.csv', index_col=0)
don_pre_new_4000 = pd.read_csv('don_pre_new_4000.csv', index_col=0)
reci_pre_new_4000_4 = pd.read_csv('reci_pre_new_4000_4.csv', index_col=0)

In [2]:
# Reading data
df = pd.read_excel('C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\final_code\\RawData-LTMP.xlsx')
recipients = df.iloc[:4000, :8].reset_index(drop=True)
donors = df.iloc[:4000, 8:].reset_index(drop=True)

In [8]:
# Step 2's functions:
## Gale-Shapley Algorithm best to recipients:
def gale_shapley_1(don_pre, reci_pre):
    don_available = {recipient: donors_names[:] for recipient in recipients_names}
    waiting_list = []
    proposals = {}
    
    while len(waiting_list) < len(recipients_names):
        for recipient in recipients_names:
            if recipient not in waiting_list:
                donor = don_available[recipient]
                best_choice = reci_pre.loc[recipient][reci_pre.loc[recipient].index.isin(donor)].idxmin()
                proposals[(recipient, best_choice)] = (reci_pre.loc[recipient][best_choice], don_pre.loc[best_choice][recipient])
        
        overlays = Counter([key[1] for key in proposals.keys()])
        
        for donor in overlays.keys():
            if overlays[donor] > 1:
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() if donor in pair}.items(), key=lambda x: x[1][1])[1:]
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _donor = copy.copy(don_available[p_to_drop[0][0]])
                    _donor.remove(p_to_drop[0][1])
                    don_available[p_to_drop[0][0]] = _donor
        
        waiting_list = [recipient[0] for recipient in proposals.keys()]
    
    return proposals

## Gale-Shapley Algorithm reversed:
def gale_shapley_2(reci_pre, don_pre):
    reci_available = {donor: recipients_names[:] for donor in donors_names}
    waiting_list = []
    proposals = {}
    
    while len(waiting_list) < len(donors_names):
        for donor in donors_names:
            if donor not in waiting_list:
                recipient = reci_available[donor]
                best_choice = don_pre.loc[donor][don_pre.loc[donor].index.isin(recipient)].idxmin()
                proposals[(donor, best_choice)] = (don_pre.loc[donor][best_choice], reci_pre.loc[best_choice][donor])
        
        overlays = Counter([key[1] for key in proposals.keys()])
        
        for recipient in overlays.keys():
            if overlays[recipient] > 1:
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() if recipient in pair}.items(), key=lambda x: x[1][1])[1:]
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _recipient = copy.copy(reci_available[p_to_drop[0][0]])
                    _recipient.remove(p_to_drop[0][1])
                    reci_available[p_to_drop[0][0]] = _recipient
        
        waiting_list = [donor[0] for donor in proposals.keys()]
    
    return proposals

# Step 3's functions:
# Check mutual exclusion and remove items that cannot be paired in any stable match
def check_and_remove_exclusivity(filtered_reci_pre, filtered_don_pre):
    # Remove donor who is not in the recipient's list of preferences
    for r, donors in filtered_reci_pre.items():
        for d in donors[:]:
            if r not in filtered_don_pre[d]:
                filtered_reci_pre[r].remove(d)

    # # Remove recipient who is not in the recipient's preference list
    for d, recipients in filtered_don_pre.items():
        for r in recipients[:]:
            if d not in filtered_reci_pre[r]:
                filtered_don_pre[d].remove(r)

    return filtered_reci_pre, filtered_don_pre

# Function that rearranges the preference list in order of the matrix
def reorder_preference(pref_dict, pref_matrix):
    reordered_dict = {}
    for key, values in pref_dict.items():
        reordered_dict[key] = sorted(values, key=lambda x: pref_matrix.loc[key, x])
    return reordered_dict

# Step 4's functions:
def find_closed_cycles(filtered_reci_pre, max_length=10):
    # Create empty lists to store cycles and closed cycles
    cycle = []
    closed_cycles = []

    # First step: Create initial lists for recipients with preference lists longer than 1
    for r in filtered_reci_pre:
        if len(filtered_reci_pre[r]) > 1:
            # Initialize the list with the recipient and their second preferred donor
            initial_list = [r, filtered_reci_pre[r][1]]
            cycle.append(initial_list)
    # print(cycle)

    # Loop until cycle is empty
    while cycle:
        new_cycle = []  # To store new lists generated in each iteration

        # Check each list in cycle
        for lst in cycle[:]:  # Use a copy of cycle to modify the original
            # First, check if the list forms a closed cycle
            if lst[0] == lst[-1]:
                closed_cycles.append(lst)  # Store the closed cycle
                cycle.remove(lst)  # Remove from cycle
            elif len(lst) > max_length:
                # Remove lists that exceed maximum allowed length
                cycle.remove(lst)
            elif len(set(lst)) < len(lst):
                # Remove lists with repeated recipients/donors (excluding the case where start and end are the same)
                cycle.remove(lst)
            else:
                if len(lst) % 2 == 1:  # If the list length is odd
                    last_recip = lst[-1]
                    if len(filtered_reci_pre[last_recip]) > 1:
                        next_donor = filtered_reci_pre[last_recip][1]  # Take the second preference
                        lst.append(next_donor)  # Extend the list
                elif len(lst) % 2 == 0:  # If the list length is even
                    last_donor = lst[-1]
                    found = False
                    # Find recipients whose first preference matches the last donor
                    for recip, preferences in filtered_reci_pre.items():
                        if preferences[0] == last_donor:
                            new_list = lst + [recip]
                            new_cycle.append(new_list)
                            found = True
                    cycle.remove(lst)  # Remove the original list
                    if not found:
                        # If no matching recipient found, do nothing (lst already removed)
                        pass

        # Add new lists to cycle
        cycle.extend(new_cycle)

    return closed_cycles

# Remove duplicate cycles
def remove_duplicates(lists):
    seen = set()
    result = []

    for lst in lists:
        # Convert the list to a tuple of tuples to make it hashable
        lst_tuple = tuple(lst)
        if lst_tuple not in seen:
            result.append(lst)
            seen.add(lst_tuple)

    return result


def remove_duplicate_cycles(cycles):
    unique_cycles = {}
    for cycle in cycles:
        # Convert the cycle to a frozenset to identify unique sets
        cycle_set = frozenset(cycle)
        if cycle_set not in unique_cycles:
            # Store the original cycle corresponding to the unique set
            unique_cycles[cycle_set] = cycle
    # Return the original cycles corresponding to the unique sets
    return list(unique_cycles.values())

# Step 5's function:
def update_matching(mu_R, output_lists, reci_pre_matrix, don_pre_matrix):
    # Creates a list to store all new matches

    matchings = []
    
    # Iterate over the lists in output_lists one by one
    for lst in output_lists:
        # Copy the current match and prepare to update
        new_mu_R = mu_R.copy()
        
        # Parses the list one by one, matching r and d in pairs
        for i in range(0, len(lst) - 1, 2):
            r = lst[i]
            d = lst[i + 1]
            
            # Gets the ordinal of r's preference for a
            r_preference = reci_pre_matrix.loc[r, d]
            
            # Gets the ordinal preference of a over r
            d_preference = don_pre_matrix.loc[d, r]
            
            # Generates a new pair and replaces the pair containing this r in the original mu_R
            new_mu_R[(r, d)] = (r_preference, d_preference)
            
            # Delete old pairs that contain r
            for key in list(new_mu_R.keys()):
                if key[0] == r and key[1] != d:
                    del new_mu_R[key]
        
        # Adds the new match to the matchings list
        matchings.append(new_mu_R.copy())
    
    return matchings

# Step 6's functions:
def combine_matchings(matchings, reci_pre_matrix):
    # Initialize the new matching
    combined_matching = {}

    # Get the list of all recipients
    recipients = reci_pre_matrix.index

    # For each recipient, select a donor from the combined matchings
    for recipient in recipients:
        worst_donor = None
        worst_preference = -1  # Higher preference value means the recipient likes the donor less
        matching_with_worst_donor = None  # To store the matching containing the worst donor

        # Iterate over all matchings
        for matching in matchings:
            for (r, d), (r_pref, _) in matching.items():
                if r == recipient and r_pref > worst_preference:
                    worst_preference = r_pref
                    worst_donor = (r, d)
                    matching_with_worst_donor = matching  # Record the matching containing this pair

        # Check if a worst donor was found and add it to the combined matching
        if worst_donor:
            combined_matching[worst_donor] = (worst_preference, matching_with_worst_donor[worst_donor][1])
        else:
            print(f"No valid donor found for recipient {recipient}")

    return combined_matching


def generate_pairwise_combinations_and_combine(matchings, reci_pre_matrix):
    all_combined_matchings = []

    # Generate all possible pairwise combinations of matchings
    for combination in itertools.combinations(matchings, 2):
        print(f"Combining 2 matchings: {combination}")
        # Combine the two matchings into a new matching
        new_matching = combine_matchings(combination, reci_pre_matrix)
        if new_matching:
            all_combined_matchings.append(new_matching)

    return all_combined_matchings

def add_unique_matchings(all_combined_matchings, stable_matching):
    # Convert each matching in stable_matching to a frozenset to compare
    stable_set = {frozenset(matching.items()) for matching in stable_matching}

    # Iterate over all_combined_matchings and check for duplicates
    for matching in all_combined_matchings:
        matching_frozenset = frozenset(matching.items())  # Convert current matching to frozenset
        if matching_frozenset not in stable_set:
            stable_matching.append(matching)  # Add the non-duplicate matching
            stable_set.add(matching_frozenset)  # Update the stable_set


def update_filtered_preference_lists(final_matching, filtered_reci_pre, filtered_don_pre, reci_pre_matrix, don_pre_matrix):
    # Update recipients' preference lists
    for (recipient, donor), (r_pref, _) in final_matching.items():
        # Get the recipient's current list of preferred donors
        filtered_donors = filtered_reci_pre[recipient]
        # Remove donors that the recipient prefers more than the matched donor
        filtered_reci_pre[recipient] = [d for d in filtered_donors if reci_pre_matrix.loc[recipient, d] >= r_pref]

    # Update donors' preference lists
    for (recipient, donor), (_, d_pref) in final_matching.items():
        # Get the donor's current list of preferred recipients
        filtered_recipients = filtered_don_pre[donor]
        # Remove recipients that the donor prefers less than the matched recipient
        filtered_don_pre[donor] = [r for r in filtered_recipients if don_pre_matrix.loc[donor, r] <= d_pref]

    return filtered_reci_pre, filtered_don_pre

def check_and_remove_exclusivity(filtered_reci_pre, filtered_don_pre):
    # Remove donors not in the donor's preference list
    for r, donors in filtered_reci_pre.items():
        for d in donors[:]:  # Use a copy to avoid modifying the list while iterating
            if r not in filtered_don_pre[d]:
                filtered_reci_pre[r].remove(d)

    # Remove recipients not in the recipient's preference list
    for d, recipients in filtered_don_pre.items():
        for r in recipients[:]:  # Use a copy to avoid modifying the list while iterating
            if d not in filtered_reci_pre[r]:
                filtered_don_pre[d].remove(r)

    return filtered_reci_pre, filtered_don_pre

# Convert the dictionary to frozenset to detect duplicates and remove duplicates
def remove_duplicate_dicts(all_new_matchings):
    unique_matchings = []
    seen = set()  # Record matches that have already occurred

    for matching in all_new_matchings:
        # Turn dictionary entries into Frozensets so you can hash and compare them
        matching_frozenset = frozenset(matching.items())
        
        # If the match does not occur, it is added to unique_matchings
        if matching_frozenset not in seen:
            unique_matchings.append(matching)
            seen.add(matching_frozenset)

    return unique_matchings

def update_stable_matching(stable_matching, mu_D):
    # Marks whether mu_D already exists in any dictionary in stable_matching
    is_duplicate = False

    # Traverse each dictionary in stable_matching
    for match in stable_matching:
        # Traverse each pair in mu_D
        for donor_recipient, value in mu_D.items():
            # Inverts mu_D's key (donor, recipient) to match stable_matching (recipient, donor)
            recipient_donor = (donor_recipient[1], donor_recipient[0])
            
            # If a matching recipient-donor pair is found, it is marked as duplicate
            if recipient_donor in match:
                is_duplicate = True
                break
        if is_duplicate:
            break
    
    # If there is no duplicate mu_D, add mu_D to stable_matching
    if not is_duplicate:
        stable_matching.append(mu_D)
    else:
        print(f'Donor stable matching, mu_D, exists in stable_matching.')

    return stable_matching

In [10]:
# Step 1: Define recipients' and donors' preference matrices

# Create the recipient preference matrix
reci_pref_matrix = reci_pre_new_4000_4
# Create the donor preference matrix
don_pref_matrix = don_pre_new_4000 


In [ ]:
# Step 2: Gale-Shapley algorithm implementation

stable_matching = []

recipients_names = reci_pref_matrix.index.tolist()
donors_names = don_pref_matrix.index.tolist()

# Find the optimal match for recipients
mu_R = gale_shapley_1(don_pref_matrix, reci_pref_matrix)
# Find an optimal match for donors
mu_D = gale_shapley_2(reci_pref_matrix, don_pref_matrix)
stable_matching.append(mu_R)

print("recipients optimal stable matching:", mu_R)
print("donors optimal matching:", mu_D)
stable_matching

In [ ]:
mu_R1 = mu_R
mu_D1 = mu_D

In [ ]:
import pickle
# Storage Path
file_path = r'C:\Users\Hongan Li\Desktop\Research Project\Code\GS+Gender\matching_mu_R1.pkl'

# Stores the matching dictionary to the pickle file
with open(file_path, 'wb') as file:
    pickle.dump(mu_R1, file)

In [ ]:
import pickle
# Storage Path
file_path = r'C:\Users\Hongan Li\Desktop\Research Project\Code\GS+Gender\matching_mu_D1.pkl'

# Stores the matching dictionary to the pickle file
with open(file_path, 'wb') as file:
    pickle.dump(mu_D1, file)

In [1]:
import pickle
file_path1 = r'C:\Users\Hongan Li\Desktop\Research Project\Code\GS+Gender\matching_mu_R1.pkl'
file_path2 = r'C:\Users\Hongan Li\Desktop\Research Project\Code\GS+Gender\matching_mu_D1.pkl'
stable_matching = []

# Load the matching dictionary from the pickle file
def load_matching(file_path):
    with open(file_path, 'rb') as file:
        mu_R_loaded = pickle.load(file)
    return mu_R_loaded

# 从Load the matching dictionary from the pickle file
def load_matching(file_path):
    with open(file_path, 'rb') as file:
        mu_D_loaded = pickle.load(file)
    return mu_D_loaded


# Invoke the loaded matching dictionary
mu_R = load_matching(file_path1)
print(mu_R)

# Invoke the loaded matching dictionary
mu_D = load_matching(file_path2)
print(mu_D)

stable_matching.append(mu_R)

{('r204', 'd2127'): (1, 34), ('r231', 'd4000'): (1, 2), ('r2254', 'd1425'): (1, 8), ('r2427', 'd786'): (1, 16), ('r2966', 'd2165'): (1, 49), ('r3244', 'd615'): (1, 5), ('r3783', 'd1'): (1, 11), ('r3853', 'd3860'): (1, 3), ('r3858', 'd391'): (1, 30), ('r3885', 'd3632'): (1, 6), ('r3944', 'd3752'): (1, 16), ('r138', 'd1266'): (2, 18), ('r221', 'd3170'): (2, 47), ('r646', 'd2016'): (2, 14), ('r1938', 'd1338'): (2, 15), ('r3395', 'd2993'): (2, 7), ('r3440', 'd3233'): (2, 7), ('r3833', 'd88'): (2, 133), ('r3934', 'd3653'): (2, 13), ('r437', 'd2935'): (3, 26), ('r711', 'd883'): (3, 12), ('r1577', 'd3532'): (3, 22), ('r1618', 'd2048'): (3, 64), ('r2464', 'd3559'): (3, 15), ('r2475', 'd631'): (3, 9), ('r3867', 'd1084'): (3, 187), ('r3990', 'd1128'): (3, 212), ('r889', 'd3260'): (4, 12), ('r1125', 'd311'): (4, 40), ('r1789', 'd1738'): (4, 45), ('r2466', 'd1024'): (4, 18), ('r2640', 'd884'): (4, 37), ('r3836', 'd706'): (4, 198), ('r729', 'd2207'): (5, 19), ('r790', 'd2984'): (5, 39), ('r1235', '

In [11]:
# Step 3: Generate updated preference sequences based on mu_R and mu_D 
# Screen eligible donors for each recipient

filtered_reci_pre = {}
for r, d in mu_R.keys():

    # Gets the ordinal of r's preference for matching donor in mu_R
    r_pref_muR = reci_pref_matrix.loc[r, d]
    
    # Obtain the donor for r matching in mu_D
    matching_donor_in_muD = [key[0] for key in mu_D.keys() if key[1] == r][0]
    r_pref_muD = reci_pref_matrix.loc[r, matching_donor_in_muD]

    # Screen eligible donors
    filtered_donors = reci_pref_matrix.columns[(reci_pref_matrix.loc[r] < r_pref_muR) | (reci_pref_matrix.loc[r] > r_pref_muD)].tolist()
    filtered_reci_pre[r] = filtered_donors

    # Remove ineligible donors
    filtered_reci_pre[r] = reci_pref_matrix.loc[r].drop(filtered_donors).index.tolist()

# Screen eligible recipients for each donor d
filtered_don_pre = {}
for d, r in mu_D.keys():
    
    # Get d's preference ordinal for the matching recipient in mu_D
    d_pref_muD = don_pref_matrix.loc[d, r]
    
    # Get the d matching recipient in mu_R
    matching_recipient_in_muR = [key[0] for key in mu_R.keys() if key[1] == d][0]
    d_pref_muR = don_pref_matrix.loc[d, matching_recipient_in_muR]

    # Filter eligible recipients
    filtered_recipients = don_pref_matrix.columns[(don_pref_matrix.loc[d] < d_pref_muD) | (don_pref_matrix.loc[d] > d_pref_muR)].tolist()
    filtered_don_pre[d] = filtered_recipients

    # Delete recipients that do not qualify
    filtered_don_pre[d] = don_pref_matrix.loc[d].drop(filtered_recipients).index.tolist()


# Call function to check mutual exclusion and update simplified preference list
filtered_reci_pre, filtered_don_pre = check_and_remove_exclusivity(filtered_reci_pre, filtered_don_pre)


# Reorder the recipient preference order
filtered_reci_pre1 = reorder_preference(filtered_reci_pre, reci_pref_matrix)
filtered_don_pre1 = reorder_preference(filtered_don_pre, don_pref_matrix)
print(filtered_reci_pre1)
print(filtered_don_pre1)

{'r204': ['d2127'], 'r231': ['d4000'], 'r2254': ['d1425'], 'r2427': ['d786'], 'r2966': ['d2165'], 'r3244': ['d615'], 'r3783': ['d1'], 'r3853': ['d3860'], 'r3858': ['d391'], 'r3885': ['d3632'], 'r3944': ['d3752'], 'r138': ['d1266'], 'r221': ['d3170'], 'r646': ['d2016'], 'r1938': ['d1338'], 'r3395': ['d2993'], 'r3440': ['d3233'], 'r3833': ['d88'], 'r3934': ['d3653'], 'r437': ['d2935'], 'r711': ['d883'], 'r1577': ['d3532'], 'r1618': ['d2048'], 'r2464': ['d3559'], 'r2475': ['d631'], 'r3867': ['d1084'], 'r3990': ['d1128'], 'r889': ['d3260'], 'r1125': ['d311'], 'r1789': ['d1738'], 'r2466': ['d1024'], 'r2640': ['d884'], 'r3836': ['d706'], 'r729': ['d2207'], 'r790': ['d2984'], 'r1235': ['d1025'], 'r2904': ['d3775'], 'r2926': ['d3530'], 'r3854': ['d397'], 'r1992': ['d2978'], 'r2031': ['d2932'], 'r2811': ['d3529'], 'r3327': ['d2025'], 'r3443': ['d3257'], 'r3900': ['d2462'], 'r1494': ['d891'], 'r1815': ['d985'], 'r2498': ['d312'], 'r3273': ['d632'], 'r3516': ['d3555'], 'r1097': ['d2974'], 'r1669'

In [12]:
# Step4：Find all closed loops according to the updated preference list filtered_reci_pre

closed_cycles = find_closed_cycles(filtered_reci_pre1)
output_lists = remove_duplicates(closed_cycles)
output_lists = remove_duplicate_cycles(output_lists)
print(output_lists)

[]


In [16]:
# Step 5: New stable matchings are generated for each closed loop

new_matching = update_matching(mu_R, output_lists, reci_pref_matrix, don_pref_matrix)
for i in new_matching:
    stable_matching.append(i)

print(new_matching)
stable_matching

[]


[{('r204', 'd2127'): (1, 34),
  ('r231', 'd4000'): (1, 2),
  ('r2254', 'd1425'): (1, 8),
  ('r2427', 'd786'): (1, 16),
  ('r2966', 'd2165'): (1, 49),
  ('r3244', 'd615'): (1, 5),
  ('r3783', 'd1'): (1, 11),
  ('r3853', 'd3860'): (1, 3),
  ('r3858', 'd391'): (1, 30),
  ('r3885', 'd3632'): (1, 6),
  ('r3944', 'd3752'): (1, 16),
  ('r138', 'd1266'): (2, 18),
  ('r221', 'd3170'): (2, 47),
  ('r646', 'd2016'): (2, 14),
  ('r1938', 'd1338'): (2, 15),
  ('r3395', 'd2993'): (2, 7),
  ('r3440', 'd3233'): (2, 7),
  ('r3833', 'd88'): (2, 133),
  ('r3934', 'd3653'): (2, 13),
  ('r437', 'd2935'): (3, 26),
  ('r711', 'd883'): (3, 12),
  ('r1577', 'd3532'): (3, 22),
  ('r1618', 'd2048'): (3, 64),
  ('r2464', 'd3559'): (3, 15),
  ('r2475', 'd631'): (3, 9),
  ('r3867', 'd1084'): (3, 187),
  ('r3990', 'd1128'): (3, 212),
  ('r889', 'd3260'): (4, 12),
  ('r1125', 'd311'): (4, 40),
  ('r1789', 'd1738'): (4, 45),
  ('r2466', 'd1024'): (4, 18),
  ('r2640', 'd884'): (4, 37),
  ('r3836', 'd706'): (4, 198),
  

In [17]:
## Step 6: Iterate steps 6(i)-6(iii) until a stable match is found that is identical to mu_D

iterations = 0
while new_matching:
    iterations += 1
    print(f"\nIteration {iterations}:")
          
    # # Step 6(i): Combine two different matchings without duplication to generate new stable matchings
    # Assuming new_matchings and reci_pref_matrix are already defined
    # Generate all new pairwise matching combinations
    new_matchings = []
    new_matchings = generate_pairwise_combinations_and_combine(new_matching, reci_pref_matrix)

    # Add unique matchings to stable_matching
    add_unique_matchings(new_matchings, stable_matching)

    # Step 6(ii): Update the list of recipients and donors' preferences with the new stable matches and generate new stable matchings
    # Initialize lists to store the updated preference lists after each update
    updated_filtered_reci_pre_list = []
    updated_filtered_don_pre_list = []
    all_new_matchings = []

    # Loop over each matching in new_matchings
    for idx, final_matching in enumerate(new_matchings):
        print(f"Processing matching {idx + 1}:")
        
        # Make deep copies of the initial preference lists to avoid modifying the originals
        reci_pre_copy = copy.deepcopy(filtered_reci_pre1)
        don_pre_copy = copy.deepcopy(filtered_don_pre1)

        # Update the preference lists using the current matching
        updated_reci_pre, updated_don_pre = update_filtered_preference_lists(
            final_matching, reci_pre_copy, don_pre_copy, reci_pref_matrix, don_pref_matrix
        )

        # Check mutual exclusivity and remove impossible pairs
        updated_reci_pre, updated_don_pre = check_and_remove_exclusivity(updated_reci_pre, updated_don_pre)

        closed_cycles = find_closed_cycles(updated_reci_pre)
        output_lists = remove_duplicates(closed_cycles)
        output_lists = remove_duplicate_cycles(output_lists)

        # If the loop is not closed, it ends
        if not output_lists:
            print("No more closed cycles found. Terminating the loop.")
            break

        per_new_matchings = update_matching(final_matching, output_lists, reci_pref_matrix, don_pref_matrix)
        for m in per_new_matchings:
            all_new_matchings.append(m)
        
        # Store the updated preference lists
        updated_filtered_reci_pre_list.append(updated_reci_pre)
        updated_filtered_don_pre_list.append(updated_don_pre)

        # Optionally, print the updated preference lists
        print(f"New matchings after matching {idx + 1}:")
        print(per_new_matchings)
        # print(f"Updated Donors Preference List after matching {idx + 1}:")
        # print(updated_don_pre)
        print("")  # Empty line for better readability

    # Remove duplicate dictionaries
    new_matchings = []
    new_matchings = remove_duplicate_dicts(all_new_matchings)
    for i in new_matchings:
        stable_matching.append(i)

    # Step 6(iii): Each entry of new_matchings is used to update the recipients' and donors' preference lists filtered_reci_pre1 
    #              and filtered_don_pre1, respectively, and generate a new stable match based on a closed cycle

    # Initialize lists to store the updated preference lists after each update
    updated_filtered_reci_pre_list = []
    updated_filtered_don_pre_list = []
    all_new_matchings = []

    # Loop over each matching in new_matchings
    for idx, final_matching in enumerate(new_matchings):
        print(f"Processing matching {idx + 1}:")
        
        # Make deep copies of the initial preference lists to avoid modifying the originals
        reci_pre_copy = copy.deepcopy(filtered_reci_pre1)
        don_pre_copy = copy.deepcopy(filtered_don_pre1)

        # Update the preference lists using the current matching
        updated_reci_pre, updated_don_pre = update_filtered_preference_lists(
            final_matching, reci_pre_copy, don_pre_copy, reci_pref_matrix, don_pref_matrix
        )

        # Check mutual exclusivity and remove impossible pairs
        updated_reci_pre, updated_don_pre = check_and_remove_exclusivity(updated_reci_pre, updated_don_pre)

        closed_cycles = find_closed_cycles(updated_reci_pre)
        output_lists = remove_duplicates(closed_cycles)
        output_lists = remove_duplicate_cycles(output_lists)
        per_new_matchings = update_matching(final_matching, output_lists, reci_pref_matrix, don_pref_matrix)
        for m in per_new_matchings:
            all_new_matchings.append(m)
        
        # Store the updated preference lists
        updated_filtered_reci_pre_list.append(updated_reci_pre)
        updated_filtered_don_pre_list.append(updated_don_pre)

        # Optionally, print the updated preference lists
        print(f"New matchings after matching {idx + 1}:")
        print(per_new_matchings)
        # print(f"Updated Donors Preference List after matching {idx + 1}:")
        # print(updated_don_pre)
        print("")  # Empty line for better readability

    # Remove duplicate dictionaries
    new_matching = []
    new_matching = remove_duplicate_dicts(all_new_matchings)
    for i in new_matching:
        stable_matching.append(i)

stable_matching = update_stable_matching(stable_matching, mu_D)
print("Find", len(stable_matching),"stable mathings.")

Donor stable matching, mu_D, exists in stable_matching.
Find 1 stable mathings.


In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1. Define the base score
base_score = 72

# 2. Define the recipient survival rate difference rule
def calculate_recipient_score(recipient):
    score_diff = 0
    # TransplantAge condition
    if recipient['TransplantAge'] <= 65:
        score_diff += 1
    else:
        score_diff -= 15
    
    # Reci_Gender condition
    if recipient['Reci_Gender'] == 'Female':
        score_diff += 4
    else:
        score_diff -= 3
    
    # MELD condition
    if recipient['MELD'] <= 6:
        score_diff += 8
    elif 6 < recipient['MELD'] <= 15:
        score_diff += 3
    elif 15 < recipient['MELD'] <= 25:
        score_diff += 1
    else:
        score_diff -= 10
    
    # Reci_loca condition
    if recipient['Reci_loca'] == 'inpt':
        score_diff -= 3
    elif recipient['Reci_loca'] == 'home':
        score_diff += 4
    elif recipient['Reci_loca'] == 'icu':
        score_diff -= 8
    
    return score_diff

# 3. Define the donor survival rate difference rule
def calculate_donor_score(donor):
    score_diff = 0
    # Donor_Age condition
    if donor['Donor_Age'] <= 50:
        score_diff += 3
    else:
        score_diff -= 6
    
    return score_diff

# 4. Define the function to calculate the score for each donor-recipient pair
def calculate_pair_score(recipient, donor):
    recipient_score = calculate_recipient_score(recipient)
    donor_score = calculate_donor_score(donor)
    return base_score + recipient_score + donor_score

# 5. Load data
recipients = df.iloc[:4000, :8].reset_index(drop=True)
donors = df.iloc[:4000, 8:].reset_index(drop=True)

# 6. Use Stochastic Gradient Descent model to predict 'ventilated' Reci_loca
# Extract training data, excluding 'ventilated' samples
train_data = recipients[recipients['Reci_loca'] != 'ventilated']
X_train = train_data[['TransplantAge', 'Reci_Gender', 'MELD']]  # Feature selection
y_train = train_data['Reci_loca']  # Target variable

# Build classification model
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Reci_Gender'])
    ], remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(max_iter=1000, tol=1e-3))
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict for 'ventilated' samples
ventilated_data = recipients[recipients['Reci_loca'] == 'ventilated']
X_test = ventilated_data[['TransplantAge', 'Reci_Gender', 'MELD']]
predicted_loca = pipeline.predict(X_test)

# Assign predicted results to ventilated samples
recipients.loc[recipients['Reci_loca'] == 'ventilated', 'Reci_loca'] = predicted_loca

# 7. Define a list to store score statistics for each match
match_stats = []

# Loop through each stable matching
for match in stable_matching:
    scores = []  # Store scores for current matching pairs

    # Loop through each pair
    for (recipient_key, donor_key), _ in match.items():
        # Extract numerical index from 'r1' and 'd1', subtracting 1 for 0-based indexing
        try:
            recipient_index = int(recipient_key[1:]) - 1  # Extract recipient index
            donor_index = int(donor_key[1:]) - 1          # Extract donor index
        except ValueError as e:
            print(f"Error parsing recipient or donor key: {recipient_key}, {donor_key}")
            continue

        # Check if indices are within valid range
        if not (0 <= recipient_index < len(recipients)) or not (0 <= donor_index < len(donors)):
            print(f"Index out of bounds: recipient_index={recipient_index}, donor_index={donor_index}")
            continue

        # Get recipient and donor data
        recipient_data = recipients.iloc[recipient_index]
        donor_data = donors.iloc[donor_index]

        # Calculate the score for this pair
        score = calculate_pair_score(recipient_data, donor_data)
        scores.append(score)

    # Calculate score statistics for the current match: max, median, mean, and variance
    max_score = np.max(scores)
    median_score = np.median(scores)
    mean_score = np.mean(scores)
    variance_score = np.var(scores)

    # Store the statistics in the list
    match_stats.append({
        'Max Score': max_score,
        'Median Score': median_score,
        'Mean Score': mean_score,
        'Variance': variance_score
    })

# Output score statistics for all matches
for idx, stats in enumerate(match_stats):
    print(f"Matching {idx+1}:")
    print(f"Max Survival Rate: {stats['Max Score']}%")
    print(f"Survival Rate Median: {stats['Median Score']}%")
    print(f"Survival Rate Mean: {stats['Mean Score']}%")
    print(f"Survival Rate Variance: {stats['Variance']}")
    print()


Matching 1:
Max Survival Rate: 92%
Survival Rate Median: 71.0%
Survival Rate Mean: 71.121%
Survival Rate Variance: 86.267859

